# Oracle Connection Template with SQL Alchemy
* install module cx_Oracle
    * OBS: both Anaconda versions (Oracle Client and cx_Oracle) must be **all** x32 or x64
* Input database username and password

In [1]:
from getpass import getpass
from os import environ
import sqlalchemy as sql
from sqlalchemy.pool import QueuePool
import pandas as pd

# Adjust encoding to UTF8
environ["NLS_LANG"] = ".AL32UTF8"

# Get username and password hiding inputs
usr = getpass(prompt='Username: ')
psw = getpass(prompt='Password: ')

# Create url to connect to Oracle using cx_oracle. Real names for host address and service name were replaced by '<>'.
url= 'oracle+cx_oracle://' + usr + ':' + psw +'@<host_address>/?service_name=<service_name>'

# Create engine to connect to url
engine = sql.create_engine(url, poolclass=QueuePool)

# Connect
conn = engine.connect()
print('Connected.')

Usuário: ········
Senha: ········
Conectado.


**Download dataframes running queries to Oracle**

In [ ]:
df1 = pd.read_sql_query("SELECT SYSDATE FROM DUAL", conn)
df2 = pd.read_sql_query("SELECT * FROM OWNER.TABLE_NAME", conn)

In [ ]:
# You may open a SQL file to read the query
SQL = open('script.sql', 'r')

# If the file has parameters (e.g. start and end date), input the dates using format function:
df = pd.read_sql_query(SQL.read().format(Start='01/03/2017',End='31/03/2017'),conn)

**Executing DDL or DML statements directly to Oracle**

In [ ]:
conn.execute(text("CREATE TABLE TMP_TABLE AS SELECT SYSDATE FROM DUAL"))

In [ ]:
# It is possible to break into multiple lines using \ or """
conn.execute("BEGIN \
EXECUTE IMMEDIATE 'DROP TABLE TMP_TABLE'; \
EXCEPTION \
WHEN OTHERS THEN \
IF SQLCODE != -942 THEN \
RAISE; \
END IF; \
END;")

**Append using pandas**

In [ ]:
# If the columns format aren't the same, use the dtype argument as below:
df.to_sql(name='tmp_table',if_exists='append',index=False,con=conn,dtype={'char_col':types.VARCHAR(length=255),
                                                                           'num_col':types.NUMERIC()})

**End session**

In [ ]:
conn.close()

**Close conection**

In [ ]:
engine.dispose()